In [19]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
file_path = './text.txt'
lines = []
with open(file_path) as f:
    lines = f.readlines()

In [3]:
def find_max_len(train_seq):
    mx_len = 0
    for i in train_seq:
        mx_len = max(mx_len, len(i))
    print(mx_len)
    return mx_len

In [4]:
numwords = 12500
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=numwords, oov_token='<OOV>')
pad = tf.keras.preprocessing.sequence.pad_sequences

In [5]:
tokenizer.fit_on_texts(lines)
train_seq = tokenizer.texts_to_sequences(lines)

In [6]:
train_seq_filtered = []
for i in train_seq:
    if len(i) >= 2:
        train_seq_filtered.append(i)

In [7]:
max_length = find_max_len(train_seq_filtered)

16


In [24]:
def split_dataset(images, labels, train_split, shuffle):
    train_images, validation_images, train_labels, validation_labels = train_test_split(images,
                                                                                        labels,
                                                                                        train_size= train_split,
                                                                                        # stratify=labels,
                                                                                        shuffle= shuffle)
    return train_images, validation_images, train_labels, validation_labels


In [8]:
train_data = []

for line in train_seq_filtered:
    for i in range(1,len(line)):
        train_data.append(line[0:i+1])

In [9]:
max_length = find_max_len(train_data)

16


In [10]:
train_seq_pad = pad(train_data, padding='pre', truncating='pre', maxlen=max_length)

In [21]:
train_x = train_seq_pad[:,:-1]
train_y = train_seq_pad[:,-1]

In [23]:
train_y

array([ 270,   36,  970, ...,   26,  132, 1829], dtype=int32)

In [25]:
# Example use case
train_x, val_x, train_y, val_y = split_dataset(images=train_x,
                                                labels=train_y,
                                                train_split=0.70,
                                                shuffle=True)


In [26]:
train_x.shape

(119918, 15)

In [27]:
embedding_dim = 100

check_point_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=f'best_model_as_per_val_accuracy.h5',  # path to save the model if the check point matches
    save_best_only=True,
    monitor='val_accuracy',  # check if it is logged as val_acc or val_accuracy
    mode='auto'  # options {min, max, auto}
)

model = tf.keras.Sequential([
        tf.keras.layers.Embedding(input_dim=numwords+1, output_dim=embedding_dim, input_shape=[max_length-1]),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
    tf.keras.layers.LSTM(8, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(numwords+1, activation='softmax')
])

loss = tf.keras.losses.SparseCategoricalCrossentropy()
optim = tf.keras.optimizers.Adam(1e-1)

model.compile(
    loss=loss,
    optimizer=optim,
    metrics=['accuracy']
)


In [28]:
history = model.fit(
    train_x,
    train_y,
    epochs=60,
    batch_size=64,
)

Epoch 1/60
1874/1874 [==============================] - 57s 29ms/step - loss: 13.2594 - accuracy: 0.0286
Epoch 2/60
 735/1874 [==========>...................] - ETA: 38s - loss: 6.9692 - accuracy: 0.0278

KeyboardInterrupt: 